# RNA 

## Imports

In [ ]:
import pandas as pd
import numpy as np
import random

from sklearn.model_selection import train_test_split
from sklearn.datasets        import make_classification

import matplotlib.pyplot as plt
import seaborn as sns

## Set-up

In [ ]:
random_state = 42
random.seed(random_state)

In [ ]:
df_original = pd.read_csv("01.df.procesado.csv")
# df_original = pd.read_csv("01.df.procesado.csv")
df_original = pd.read_csv('https://raw.githubusercontent.com/blukitas/AA-2021/main/TPs/02.TP/01.df.procesado.csv')
df_original.head()

In [ ]:
num_columns = [col for col in df_original.columns if df_original[col].dtype != "object"]
num_columns

In [ ]:
drop_columns = [
    "file_path",
    "Unnamed: 0",
    "modality",
    "vocal_channel",
#     "emotion",
    "emotional_intensity",
    "statement",
    "repetition",
    "actor",
]
num_columns = [x for x in num_columns if x not in drop_columns]
num_columns

In [ ]:
df_proc = df_original[num_columns] #.head(10000)

## Split

In [ ]:
# X, y = make_classification(n_samples=1000, n_features=4, n_classes=2, random_state=random_state)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df_proc.loc[:, df_proc.columns != "emotion"],
    df_proc["emotion"],
    stratify=df_proc["emotion"],
    random_state=66,
)


## Adrian notebook

Ejemplo que paso adrián mostranod un poco las funciones de activacion

### Funcs activacion

In [ ]:
# Escalon
binary_step = lambda z: 1.0 if z >= 0 else 0.0

parametric_relu = lambda alpha: lambda z: np.maximum(0, z) * alpha

relu = parametric_relu(alpha=1)

leaky_relu = parametric_relu(alpha=0.01)

parametric_elu = lambda alpha: lambda z:  alpha * (np.exp(z) -1) if z < 0 else z

elu = parametric_elu(alpha=1.0)

sigmoid = lambda z: 1 / (1 + np.exp(-z))

identity = lambda z: z

tanh = lambda z: (2 / (1 + np.exp(-2 * z))) -1

arc_tan = lambda z: np.arctan(z)

Metrica optimizar -> MSE

In [ ]:
mse = lambda y_true, x_true: sum((y_true - x_true) ** 2) / len(x_true)

In [ ]:
class Perceptron:
    def __init__(self, act_fn, inputs_count):
        self.act_fn = act_fn
        # Initilizamos los pesos con valores random...
        self.weights = [random.random() for _ in range(0, inputs_count)]

    def predict(self, features):
        output = sum([w * f for w, f in zip(self.weights, features)])
        return self.act_fn(output)

    def fit(
        self, 
        X_train, y_train, X_val, y_val, 
        metric_fn, 
        learning_rate, 
        epocs,
        verbose=False
    ):
        summary = []

        # Repetimos el proceso de entenamiento "epocs" veces...
        for epoc in range(1, epocs -1):
            
            # Ajustamos los pesos del modelo...
            for features, y_true in zip(X_train, y_train):
                y_pred = self.predict(features)
                self.__back_propagation(features, y_pred, y_true, learning_rate)

            # Calculamos la metrica para el conjunto de validacion y train
            metric_values = self.__calculate_metric(
                metric_fn, 
                X_train, y_train,  
                X_val, y_val
            )
            if verbose:
                print(f'epoc: {epoc}, train: {metric_values[0]}, val: {metric_values[1]}')
            
            summary.append(metric_values)

        return summary

    def __back_propagation(self, features, y_pred, y_true, learning_rate):
        # Ajustamos los pesos del modelo, dado un ejemplo a aprender...
        for index, feature in enumerate(features):
            self.weights[index] += learning_rate * (y_true - y_pred) * feature

    def __calculate_metric(self, metric_fn, X_train, y_train, X_val, y_val):
        train_metric_value = metric_fn([self.predict(x) for x in X_train], y_train)                                                                                    
        val_metric_value   = metric_fn([self.predict(x) for x in X_val], y_val)
        return train_metric_value, val_metric_value


Funcion para graficar train vs validation:

In [ ]:
def plot_metrics(summary):
    sns.set_style("darkgrid")
    sns.lineplot(data=[summary[i][0] for i in range(0, len(summary))], label='Train')
    sns.lineplot(data=[summary[i][1] for i in range(0, len(summary))], label='Validation')
    plt.xlabel("Epocs")
    plt.ylabel("MSE")
    plt.title("MSE: Train vs. Validation")
    plt.show()

Objeto que permite realizar corridas del modelo con distintos hiper-parametros y ademas grafica las curvas de evolucion de la metrica elegida (MSE) en train y evaluation: 

In [ ]:
class ModelTest:
    def __init__(self, X_train, y_train, X_val, y_val):
        self.X_train = X_train
        self.y_train = y_train
        self.X_val   = X_val
        self.y_val   = y_val
        # La cantidad de entradas es igual a la cantidad de features...
        self.inputs_count = len(X_train[0])

    def perform(
        self,
        # Function de activacion
        act_fn, 
        metric_fn     = mse,
        learning_rate = 0.00001, 
        epocs         = 350,
        verbose       = False
    ):
        model = Perceptron(act_fn, self.inputs_count)

        summary = model.fit(
            self.X_train,
            self.y_train,
            self.X_val, 
            self.y_val,
            metric_fn,
            learning_rate, 
            epocs,
            verbose
        )

        plot_metrics(summary)

        print('Val MSE:', summary[-1][1])        

In [ ]:
test = ModelTest(X_train, y_train, X_val, y_val)

### Aprender

In [ ]:
# Aprendizaje usando la funcion de activación escalon:
test.perform(act_fn = binary_step)

In [ ]:
# Aprendizaje usando la funcion de activación Sigmoide:
test.perform(act_fn = sigmoid, epocs = 3_000)

In [ ]:
# Aprendizaje usando la funcion de activación ReLU:
test.perform(act_fn = relu)

In [ ]:
# Aprendizaje usando la funcion de activación Leaky-ReLU:
test.perform(act_fn = leaky_relu)

In [ ]:
# Aprendizaje usando la funcion de activación Parametric-ReLU:
test.perform(act_fn = parametric_relu(alpha = 0.05))

In [ ]:
# Aprendizaje usando la funcion de activación Identidad:
test.perform(act_fn = identity)

In [ ]:
# Aprendizaje usando la funcion de activación tangente hiperbólica:
test.perform(act_fn = tanh)

In [ ]:
# Aprendizaje usando la funcion de activación ELU:
test.perform(act_fn = elu)

In [ ]:
# Aprendizaje usando la funcion de activación Parametric-ELU:
test.perform(act_fn = parametric_elu(alpha = 0.001))

In [ ]:
# Aprendizaje usando la funcion de activación arcotangente:
test.perform(act_fn = arc_tan)

## RNA - Siguiendo clase


In [ ]:
print('Dimensiones: {}'.format(X_train.shape))

In [ ]:
layer_in = tfkl.Input(shape=(28,28)) #Todo modelo necesita una entrada, y debemos especificar sus dimensiones
#Esta capa rompe la estructura de imagen y nos deja un vector (1x784)
flatten_layer = tfkl.Flatten()(layer_in)
#Esta es la capa de salida
hidden_layer = tfkl.Dense(units=128, activation='relu')(flatten_layer)
#Esta es la capa de salida:
output_layer = tfkl.Dense(units=10,activation='softmax')(hidden_layer)  

In [ ]:
mlp_model = tf.keras.Model(inputs=[layer_in],outputs=[output_layer])

In [ ]:
mlp_model.summary()

In [ ]:
tf.keras.utils.plot_model(mlp_model,show_shapes=True)

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
metrics_list = [tf.keras.metrics.CategoricalAccuracy(),
                tf.keras.metrics.Precision(),
                tf.keras.metrics.Recall(),
                tf.keras.metrics.AUC()]

mlp_model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=metrics_list)

Por último, hay que pasarle al modelo los datos de entrenamiento.

Primero abrimos tensorboard, el cual nos permite monitorear la performance del modelo durante el entrenamiento.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir 'tblogs'

Es posible especificar funciones o callbacks que seran llamadas durante el entrenamiento del modelo. En este caso usamos:

* **TensorBoard**: guarda métricas y logs del entrenamiento para su visualización en TensorBoard.
* **ModelCheckpoint**: guarda los parámetros del modelo al finalizar cada época de entrenamiento, lo cual es útil para restaurar modelos o retomar el entrenamiento si se interrumpe.
* **EarlyStopping**: es una estrategia para evitar el sobre-entrenamiento, la cual consiste en dejar de entrenar el modelo si el error de validación deja de disminuir. En este caso, monitor es la métrica que vamos a analizar para decidir cuando deja de mejorar el modelo, y patience indica cuántas épocas sin mejorar el mejor modelo hay que esperar antes de detener el entrenamiento.

In [ ]:
cb_list = [tf.keras.callbacks.TensorBoard(log_dir='tblogs'),
             tf.keras.callbacks.ModelCheckpoint(filepath='checkpoints'),
          tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5)]

mlp_model.fit(x=train_images,y=train_labels,validation_data=(val_images,val_labels),batch_size=128,epochs=50,callbacks=cb_list)

Podemos buscar el tamaño óptimo de la capa oculta:

In [ ]:
from sklearn.model_selection import ParameterSampler

hyperparameter_space = {'units': np.arange(10,1000,20),'activation': ['relu','tanh','sigmoid','elu'],'batch_size':[16,32,64,128,256,512]}
hyperparameters = list(ParameterSampler(hyperparameter_space,n_iter=10))

Para emprolijar las cosas puedo ahora armar una función con todo lo que hecho:

In [ ]:
def entrenar_modelo(hparams):
    layer_in = tfkl.Input(
        shape=(28, 28)
    )  # Todo modelo necesita una entrada, y debemos especificar sus dimensiones
    # Esta capa rompe la estructura de imagen y nos deja un vector (1x784)
    flatten_layer = tfkl.Flatten()(layer_in)
    # Esta es la capa de salida
    hidden_layer = tfkl.Dense(units=hparams["units"], activation=hparams["activation"])(
        flatten_layer
    )
    # Esta es la capa de salida:
    output_layer = tfkl.Dense(units=10, activation="softmax")(hidden_layer)

    mlp_model = tf.keras.Model(inputs=[layer_in], outputs=[output_layer])

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    metrics_list = [
        tf.keras.metrics.CategoricalAccuracy(),
        tf.keras.metrics.Precision(),
        tf.keras.metrics.Recall(),
        tf.keras.metrics.AUC(),
    ]
    metric_names = ["loss", "acc", "precision", "recall", "auc"]
    mlp_model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=metrics_list
    )

    cb_list = [
        tf.keras.callbacks.TensorBoard(log_dir="tblogs"),
        tf.keras.callbacks.ModelCheckpoint(filepath="checkpoints"),
        tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5),
    ]

    mlp_model.fit(
        x=train_images,
        y=train_labels,
        validation_data=(val_images, val_labels),
        batch_size=hparams["batch_size"],
        epochs=50,
        callbacks=cb_list,
    )
    performance = mlp_model.evaluate(x=val_images, y=val_labels)
    performance = dict(zip(metric_names, performance))

    return mlp_model, performance, hparams

In [ ]:
best_recall = 0
best_model = None
best_params = None
hyp_results = []
for hparam in hyperparameters:
    model, perf, hparams = entrenar_modelo(hparam)
    if perf["recall"] > best_recall:
        best_recall = perf["recall"]
        best_model = model
        best_params = hparams
    hyp_results.append({"params": hparams, "performance": perf})

In [ ]:
best_params

Gráficar parametros - performance

In [ ]:
x = "activation"
y = "auc"

fig, ax = plt.subplots(nrows=5, ncols=3, figsize=(15, 10))
x_labels = ["activation", "batch_size", "units"]
y_labels = ["acc", "auc", "loss", "precision", "recall"]
for i, y in enumerate(y_labels):
    for j, x in enumerate(x_labels):
        xs = []
        ys = []
        for run in hyp_results:
            xs.append(run["params"][x])
            ys.append(run["performance"][y])
        if i == len(y_labels) - 1:
            ax[i, j].set_xlabel(x)
        if j == 0:
            ax[i, j].set_ylabel(y)
        ax[i, j].scatter(xs, ys, alpha=0.6, c="r")

In [ ]:
test_performance = best_model.evaluate(x=test_images,y=test_labels)
metric_names = ['loss','acc','precision','recall','auc']
test_performance = dict(zip(metric_names,test_performance))

In [ ]:
test_performance